In [3]:
from faker import Faker
import pandas as pd
import random

# Initialize Faker (US locale for realistic city/state names)
fake = Faker('en_US')

rows = []
target_rows = 100000  # 1 lakh

for _ in range(target_rows):
    # Generate name
    name = fake.name().split()
    first_name = name[0]
    last_name = name[-1]

    # Generate gender (using Faker profile gives "M" / "F")
    gender = random.choice(['Male', 'Female'])

    # Generate DOB between 1950 and 2010 (customizable)
    date_of_birth = fake.date_of_birth(minimum_age=15, maximum_age=95)

    # Generate city and state
    city = fake.city()
    state = fake.state()

    rows.append([first_name, last_name, gender, date_of_birth, city, state])

# Create DataFrame
df = pd.DataFrame(rows, columns=["first_name", "last_name", "gender", "date_of_birth", "city", "state"])

# Save to CSV
df.to_csv("synthetic_people_100k.csv", index=False)

print("✅ Dataset created: synthetic_people_100k.csv with", len(df), "rows")


✅ Dataset created: synthetic_people_100k.csv with 100000 rows


In [1]:
import pandas as pd
import random
import string

# Load existing CSV
df = pd.read_csv("synthetic_people_100k.csv")

# Generate unique 6-char alphanumeric IDs
ids = set()
while len(ids) < len(df):
    new_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    if new_id not in ids:
        ids.add(new_id)

df['patient_id'] = list(ids)

# Reorder columns so patient_id comes first
df = df[['patient_id','first_name','last_name','gender','date_of_birth','city','state']]

# Save new CSV
df.to_csv("synthetic_patients_100k.csv", index=False)
print("✅ CSV updated with unique alphanumeric patient_id")


✅ CSV updated with unique alphanumeric patient_id


In [2]:
import csv
import random
import string
import oracledb

# -------------------------------
# 1. Database Connection
# -------------------------------
connection = oracledb.connect(
    user="project_user",
    password="Chethan@28",
    dsn="localhost:1521/freepdb1"
)

cursor = connection.cursor()

# -------------------------------
# 2. Function to generate unique patient_id
# -------------------------------
def generate_patient_id(existing_ids):
    while True:
        pid = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
        if pid not in existing_ids:
            existing_ids.add(pid)
            return pid

# -------------------------------
# 3. Read CSV and Insert
# -------------------------------
existing_ids = set()
rows = []

with open("synthetic_patients_100k.csv", "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        patient_id = generate_patient_id(existing_ids)
        rows.append((
            patient_id,
            row["first_name"],
            row["last_name"],
            row["gender"],
            row["date_of_birth"],   # should be YYYY-MM-DD in CSV
            row["city"],
            row["state"]
        ))

# -------------------------------
# 4. Bulk Insert
# -------------------------------
cursor.executemany("""
    INSERT INTO patients
    (patient_id, first_name, last_name, gender, date_of_birth, city, state)
    VALUES (:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'), :6, :7)
""", rows)

connection.commit()
print(f"✅ Inserted {len(rows)} records into patients table.")

# -------------------------------
# 5. Close
# -------------------------------
cursor.close()
connection.close()


✅ Inserted 100000 records into patients table.


In [1]:
import oracledb
import random

# Connect as project_user
conn = oracledb.connect(
    user="project_user",
    password="Chethan@28",
    dsn="localhost:1521/freepdb1"
)
cursor = conn.cursor()

# 1. Get all doctor_ids
cursor.execute("SELECT doctor_id FROM doctors")
doctor_ids = [row[0] for row in cursor.fetchall()]

# 2. Get all patients (just their IDs)
cursor.execute("SELECT patient_id FROM patients")
patient_ids = [row[0] for row in cursor.fetchall()]

# 3. Assign random doctor_id to each patient
updates = []
for pid in patient_ids:
    random_doc = random.choice(doctor_ids)  # not equal distribution
    updates.append((random_doc, pid))

# 4. Bulk update
cursor.executemany("UPDATE patients SET doctor_id = :1 WHERE patient_id = :2", updates)

# 5. Commit changes
conn.commit()
cursor.close()
conn.close()

print(f"✅ Updated {len(updates)} patients with random doctors")


✅ Updated 100000 patients with random doctors


In [9]:
import oracledb

conn = oracledb.connect(
    user="project_user",
    password="Chethan@28",
    dsn="localhost:1521/freepdb1"
)
cursor = conn.cursor()

# 0. Drop leftover temp table if exists
try:
    cursor.execute("DROP TABLE patients_new CASCADE CONSTRAINTS")
except oracledb.DatabaseError:
    pass  # ignore if doesn't exist

# 0. Drop leftover constraints if already exist on patients
for cons in ["pk_patients_new", "fk_patients_new_doctor"]:
    try:
        cursor.execute(f"ALTER TABLE patients DROP CONSTRAINT {cons}")
    except oracledb.DatabaseError:
        pass  # ignore if doesn't exist

# 1. Find max length of patient_id
cursor.execute("SELECT MAX(LENGTH(patient_id)) FROM patients")
max_len = cursor.fetchone()[0]
if max_len is None:
    max_len = 10

print(f"Max patient_id length = {max_len}")

# 2. Create new table
cursor.execute(f"""
    CREATE TABLE patients_new (
        patient_id    VARCHAR2({max_len}),
        doctor_id     NUMBER,
        first_name    VARCHAR2(50),
        last_name     VARCHAR2(50),
        gender        VARCHAR2(10),
        date_of_birth DATE,
        city          VARCHAR2(50),
        state         VARCHAR2(50),
        CONSTRAINT pk_patients_new PRIMARY KEY (patient_id),
        CONSTRAINT fk_patients_new_doctor FOREIGN KEY (doctor_id)
            REFERENCES doctors(doctor_id)
    )
""")

# 3. Copy data
cursor.execute("""
    INSERT INTO patients_new (
        patient_id, doctor_id, first_name, last_name,
        gender, date_of_birth, city, state
    )
    SELECT patient_id, doctor_id, first_name, last_name,
           gender, date_of_birth, city, state
    FROM patients
""")

# 4. Drop old patients
cursor.execute("DROP TABLE patients CASCADE CONSTRAINTS")

# 5. Rename new table
cursor.execute("ALTER TABLE patients_new RENAME TO patients")

# 6. Rename constraints
cursor.execute("ALTER TABLE patients RENAME CONSTRAINT pk_patients_new TO pk_patients")
cursor.execute("ALTER TABLE patients RENAME CONSTRAINT fk_patients_new_doctor TO fk_patients_doctor")

# Commit
conn.commit()
cursor.close()
conn.close()

print("✅ Recreated patients table with doctor_id as 2nd column")


Max patient_id length = 6
✅ Recreated patients table with doctor_id as 2nd column


ERROR LOOP HAS NO END 

In [ ]:
'''
import random
import oracledb
from datetime import datetime, timedelta

# connect as project_user
conn = oracledb.connect(
    user="project_user",
    password="Chethan@28",
    dsn="localhost:1521/freepdb1"
)
cursor = conn.cursor()

# medicine mapping by specialization
specialization_medicines = {
    'Cardiology': ["Aspirin", "Atorvastatin", "Metoprolol"],
    'Neurology': ["Levetiracetam", "Carbamazepine", "Gabapentin"],
    'Orthopedics': ["Ibuprofen", "Diclofenac", "Naproxen"],
    'Dermatology': ["Clobetasol", "Hydrocortisone", "Cetirizine"],
    'Oncology': ["Cyclophosphamide", "Methotrexate", "Paclitaxel"],
    'Gynecology': ["Clomiphene", "Progesterone", "Estradiol"],
    'Pediatrics': ["Amoxicillin", "Paracetamol", "Salbutamol"],
    'Psychiatry': ["Sertraline", "Olanzapine", "Diazepam"],
    'General Medicine': ["Vitamin D", "Iron Supplement", "Paracetamol"],
    'Endocrinology': ["Insulin", "Metformin", "Levothyroxine"],
    'ENT': ["Cetirizine", "Amoxicillin", "Pseudoephedrine"],
    'Pulmonology': ["Salbutamol", "Budesonide", "Montelukast"],
    'Nephrology': ["Furosemide", "Spironolactone", "Erythropoietin"],
    'Urology': ["Tamsulosin", "Finasteride", "Oxybutynin"]
}

# possible dosages
dosages = ["1 tablet", "2 tablets", "250mg", "500mg", "5ml", "10ml", "1/day", "2/day", "3/day"]

# fetch patients joined with doctors (patient_id, doctor_id, specialization, dob)
cursor.execute("""
    SELECT p.patient_id, p.doctor_id, d.specialization, p.date_of_birth
    FROM patients p
    JOIN doctors d ON p.doctor_id = d.doctor_id
""")
patient_data = cursor.fetchall()

if not patient_data:
    raise Exception("No patient-doctor data found!")

# target = 24,267 rows
target_rows = 24267
batch_size = 1000
insert_sql = """
    INSERT INTO prescriptions (prescription_id, patient_id, doctor_id, medicine_name, dosage, prescription_date, cost)
    VALUES (:1, :2, :3, :4, :5, :6, :7)
"""

prescriptions = []
used_ids = set()

while len(prescriptions) < target_rows:
    patient_id, doctor_id, specialization, dob = random.choice(patient_data)

    # ensure unique 6-digit prescription_id
    while True:
        prescription_id = random.randint(100000, 999999)
        if prescription_id not in used_ids:
            used_ids.add(prescription_id)
            break

    # choose medicine by specialization
    medicines = specialization_medicines.get(specialization, ["Paracetamol"])
    medicine_name = random.choice(medicines)

    # choose dosage
    dosage = random.choice(dosages)

    # prescription date: at least 20 years after DOB
    if dob:
        min_date = dob + timedelta(days=365 * 20)
        max_date = datetime.now()
        if min_date < max_date:
            random_days = random.randint(0, (max_date - min_date).days)
            prescription_date = min_date + timedelta(days=random_days)
        else:
            prescription_date = max_date
    else:
        prescription_date = datetime.now()

    # cost between 501.18 and 4999.92
    cost = round(random.uniform(501.18, 4999.92), 2)

    prescriptions.append((
        prescription_id, patient_id, doctor_id, medicine_name, dosage, prescription_date, cost
    ))

    # batch insert
    if len(prescriptions) % batch_size == 0:
        cursor.executemany(insert_sql, prescriptions)
        conn.commit()
        prescriptions = []

# insert any leftover
if prescriptions:
    cursor.executemany(insert_sql, prescriptions)
    conn.commit()

cursor.close()
conn.close()

print("✅ Inserted 24,267 prescriptions into prescriptions table.")
'''

In [1]:
import random
import oracledb

# connect as project_user
conn = oracledb.connect(
    user="project_user",
    password="Chethan@28",
    dsn="localhost:1521/freepdb1"
)
cursor = conn.cursor()

# fetch prescription data (patient_id, cost) to align billing rows
cursor.execute("SELECT patient_id, cost FROM prescriptions ORDER BY prescription_id")
prescription_data = cursor.fetchall()

# payment modes
payment_modes = ["online mode", "cash deposit", "below poverty line FREE"]

# insert SQL
insert_sql = """
    INSERT INTO billing (bill_id, patient_id, total_amount, payment_mode)
    VALUES (:1, :2, :3, :4)
"""

billing_rows = []
used_ids = set()

for patient_id, cost in prescription_data:
    # unique 6-digit bill_id
    while True:
        bill_id = random.randint(100000, 999999)
        if bill_id not in used_ids:
            used_ids.add(bill_id)
            break

    # add random amount (less uniform)
    extra_cost = random.randint(501, 1999)
    if random.random() < 0.5:  # add fractional variation sometimes
        extra_cost += round(random.random(), 2)

    total_amount = round(cost + extra_cost, 2)

    # choose random payment mode
    payment_mode = random.choice(payment_modes)

    billing_rows.append((bill_id, patient_id, total_amount, payment_mode))

# batch insert
cursor.executemany(insert_sql, billing_rows)
conn.commit()

cursor.close()
conn.close()

print(f"✅ Inserted {len(billing_rows)} rows into billing table.")

✅ Inserted 24266 rows into billing table.
